After looking at the data, I came to the conclusion that there were far too many openings with the same opening moves (over 3000). Instead of looking at the opening name, I decided it was best to look at white's first move. I kept black's first move as I think a response to white would also bring in some interesting data. 

The issue that came up while attempting to extract the move data, is that not every recorded game had a move. Some lost due to abandonment prior to the game beginning. In terms of collecting data, I could not run the same code as before as doing so, would skip the games that did not have moves played and would move the indexes of the games all around as to make it impossible to reference. 

The first solution I came up with was to match the opening names to a library of openings and bring the moves along with it. Unfortunately, the data provided by lichess did not match entirely and over 300 opening names differed between the data set and the opening library resulting in about 2 million incomplete rows of data. So back to the drawing board. 

The solution that I came up with was to write code telling the program to look at the beginning of a game and to look for moves before the following game started. If moves were not present in that specific game, the program would put “None” in its place. I also pulled the "Site" (`Game URL`) information to be able to merge it with the larget database later on. Dealing with 200+ gb of data and doing the above became extremely memory intensive and would crash my computer (several times). The solution is the code that follows. I split the data up into smaller chunks (10 million games per file), made dataframes of the data, and then used `pd.concat` to combine these numerous csv files. This was done through several notebooks, but for simplicity, I will combine below. 


__Notebook 1: Dividing the data up into multiple files.__

In [ ]:
# The "os" module provides a way of using operating system dependent functionality like reading or writing to the file system, manipulating paths, etc. 
import os

In [ ]:
# pgn_file is the source file
pgn_file = r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\lichess_db_standard_rated_2022_08.pgn'
# game_per_file is how many games per file before the code tells the program to create a new target file.
games_per_file = 10000000 # set the number of games to write per file
# output_dir is the directory where these files will be stored. 
output_dir = r"C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\split pgn"

In [ ]:
with open(pgn_file) as f:
    game_lines = []
    game_count = 0
    file_count = 0
    for line in f:
        if line.startswith("[Event"):
            if game_lines:
                game_count += 1
                if game_count % games_per_file == 1:
                    output_file = os.path.join(output_dir, f'games_{file_count}.pgn')
                    with open(output_file, 'w') as outfile:
                        outfile.write("".join(game_lines))
                    game_lines = []
                    file_count += 1
            if game_count >= 92670441:  # set the total number of games to read
                break
        game_lines.append(line)
    if game_lines:
        output_file = os.path.join(output_dir, f'games_{file_count}.pgn')
        with open(output_file, 'w') as outfile:
            outfile.write("".join(game_lines))

__Notebook 2: Pulling First Move and URL data from files__

In [ ]:
import re
import pandas as pd

In [ ]:
lichess_data = r"C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\....txt"
# Loaded data from individual files created by above notebook.

In [ ]:
'''Used the below function to search for a string of information (`quote_pattern`) 
found on the same line but after the words '[Site'.  Put these into a list `game_urls`. '''

def get_quoted_value(source_string):
    quote_pattern = '\"(.*?)\"'
    match = re.search(quote_pattern, line).group(1)
    return match

file = open(lichess_data, mode="r")

game_urls = []

for line in file:

    if line.startswith('[Site'):
        game_urls.append(get_quoted_value(line))

In [ ]:
'''This function divides games up individually by the presence of '[Event'. 
Then, it looks for the first incident of '1.' and pulled the data on the 100 
characters following it. That many characters were pulled because some games 
included computer evaluations and clock information. That, along with a buffer, 
allowed me to make sure I pulled all the needed information. Any additional 
unwanted information, I would clean up. All of this was put into a list called `first_move`'''

def get_quoted_value(game_string):
    quote_pattern = r'1\. (.{1,100})'
    match = re.search(quote_pattern, game_string)
    return match if match else None

def games_from_file(filename):
    with open(filename) as f:
        game_lines = []
        for line in f:
            if line.startswith("[Event"):
                if game_lines:
                    yield "".join(game_lines)
                    game_lines = []
            game_lines.append(line)
        if game_lines:
            yield "".join(game_lines)

first_move = []
for game in games_from_file(lichess_data):
    first_move.append(get_quoted_value(game))

In [ ]:
# Here we spot checked the data to make sure it matched. 

print("Observations of first move:", len(first_move))
print("First instance of first move:", first_move[0])
print("Observations of game urls:", len(game_urls))
print("First instance of game url:", game_urls[0])

'''Observations of first move: 50
First instance of first move: <re.Match object; span=(369, 472), match='1. e4 { [%clk 0:01:00] } 1... g6 { [%clk 0:01:00]>
Observations of game urls: 50
First instance of game url: https://lichess.org/aAcMUViA'''

In [ ]:
df = pd.DataFrame(
    {
        'First Move': first_move, 'Game URL': game_urls
    }
)

In [ ]:
df.head(50) #We've got some cleaning up to do. 
'''
 	First Move	Game URL
0	<re.Match object; span=(369, 472), match='1. e...	https://lichess.org/aAcMUViA
1	<re.Match object; span=(390, 493), match='1. e...	https://lichess.org/brBOPdLj
2	<re.Match object; span=(383, 486), match='1. g...	https://lichess.org/pk0m4lj8
3	<re.Match object; span=(384, 487), match='1. e...	https://lichess.org/ZXitDcRa
4	<re.Match object; span=(397, 500), match='1. e...	https://lichess.org/j50FcHUm
5	<re.Match object; span=(384, 487), match='1. d...	https://lichess.org/p4EhmzYV
6	<re.Match object; span=(405, 508), match='1. N...	https://lichess.org/Z7H9A6pU
7	<re.Match object; span=(362, 465), match='1. e...	https://lichess.org/28cBdakU
8	<re.Match object; span=(384, 487), match='1. e...	https://lichess.org/JUDr2nbm
9	<re.Match object; span=(375, 478), match='1. d...	https://lichess.org/J9MxhpAQ
10	<re.Match object; span=(400, 503), match='1. d...	https://lichess.org/BGJcMxG1
11	<re.Match object; span=(367, 470), match='1. d...	https://lichess.org/9hQF2sHz
12	<re.Match object; span=(379, 482), match='1. d...	https://lichess.org/YqBei5k7
13	<re.Match object; span=(390, 493), match='1. e...	https://lichess.org/bVgQVZ7L
14	<re.Match object; span=(399, 502), match='1. e...	https://lichess.org/2ORvpVTt
15	<re.Match object; span=(410, 513), match='1. d...	https://lichess.org/rNNyi2Xh
16	<re.Match object; span=(386, 489), match='1. d...	https://lichess.org/cLhZKY8t
17	<re.Match object; span=(384, 487), match='1. e...	https://lichess.org/aVEa6ISS
18	<re.Match object; span=(388, 491), match='1. e...	https://lichess.org/vdSeE91t
19	<re.Match object; span=(395, 498), match='1. d...	https://lichess.org/yAst78o6
20	<re.Match object; span=(390, 493), match='1. e...	https://lichess.org/EXVNv52f
21	<re.Match object; span=(366, 469), match='1. e...	https://lichess.org/GL6ddml3
22	<re.Match object; span=(366, 469), match='1. e...	https://lichess.org/ckQWB3XK
23	<re.Match object; span=(375, 478), match='1. c...	https://lichess.org/gr7XA1AX
24	<re.Match object; span=(385, 488), match='1. e...	https://lichess.org/fVUOEiV4
25	<re.Match object; span=(366, 469), match='1. e...	https://lichess.org/dtat7uKG
26	<re.Match object; span=(380, 483), match='1. d...	https://lichess.org/7XCXdk7U
27	<re.Match object; span=(388, 491), match='1. c...	https://lichess.org/sN4MqZmZ
28	<re.Match object; span=(372, 475), match='1. e...	https://lichess.org/H0LxZAdI
29	<re.Match object; span=(356, 459), match='1. e...	https://lichess.org/tXk05aki
30	<re.Match object; span=(386, 489), match='1. c...	https://lichess.org/Y02JnbBc
31	<re.Match object; span=(395, 498), match='1. c...	https://lichess.org/DtZzhvxk
32	<re.Match object; span=(417, 520), match='1. e...	https://lichess.org/QYKbU5Dd
33	<re.Match object; span=(366, 469), match='1. e...	https://lichess.org/12PHsoUL
34	<re.Match object; span=(361, 464), match='1. d...	https://lichess.org/XTIQpN7V
35	<re.Match object; span=(389, 492), match='1. e...	https://lichess.org/OqxvE71k
36	<re.Match object; span=(387, 490), match='1. d...	https://lichess.org/iGnPqb95
37	<re.Match object; span=(387, 490), match='1. d...	https://lichess.org/enacEmlu
38	<re.Match object; span=(376, 479), match='1. e...	https://lichess.org/xSDUBVPU
39	<re.Match object; span=(368, 471), match='1. e...	https://lichess.org/QTeIYiUO
40	<re.Match object; span=(395, 498), match='1. e...	https://lichess.org/aWpqQoyO
41	<re.Match object; span=(395, 498), match='1. g...	https://lichess.org/sRONPg2H
42	<re.Match object; span=(370, 473), match='1. d...	https://lichess.org/pQHjr0aM
43	<re.Match object; span=(363, 466), match='1. e...	https://lichess.org/dthjBLOu
44	<re.Match object; span=(380, 483), match='1. d...	https://lichess.org/5HW80dFF
45	<re.Match object; span=(364, 467), match='1. e...	https://lichess.org/TD3hNUNX
46	<re.Match object; span=(393, 496), match='1. e...	https://lichess.org/hbEhPPu6
47	<re.Match object; span=(412, 515), match='1. N...	https://lichess.org/L42Fe6eD
48	<re.Match object; span=(365, 468), match='1. d...	https://lichess.org/VNq1r3Qu
49	<re.Match object; span=(402, 505), match='1. e...	https://lichess.org/CXsfIcQc'''

In [ ]:
# Searched and removed '\{.*?\}'

df['First Move'] = df['First Move'].astype(str)
pattern = re.compile(r'\{.*?\}')

# apply the pattern to the 'moves' column and replace with an empty string
df['First Move'] = df['First Move'].apply(lambda x: pattern.sub('', x))

print(df['First Move'])

In [ ]:
# Split first move in order to eventually remove "<re.Match object; span=(###, ###), match='1"

df[['Move Before Match', 'Match and After']] = df['First Move'].str.split("match='1. ", n=1, expand=True)
df = df[["Game URL","Match and After"]]
df = df[["Game URL","First Move"]]
df.head

'''
Game URL	First Move
0	https://lichess.org/aAcMUViA	e4 1... g6 { [%clk 0:01:00]>
1	https://lichess.org/brBOPdLj	e4 1... d5 { [%clk 0:01:00]>
2	https://lichess.org/pk0m4lj8	g3 1... c6 { [%clk 0:01:00]>
3	https://lichess.org/ZXitDcRa	e4 1... d6 { [%clk 0:01:00]>
4	https://lichess.org/j50FcHUm	e4 1... e5 { [%clk 0:01:00]>
'''

In [ ]:
# Searched for "1...", which indicates black's first move and removed the number but kept the move.

df['First Move'] = df['First Move'].str.replace('1\.\.\. ', '')
df.head()
'''	Game URL	First Move
0	https://lichess.org/aAcMUViA	e4 g6 { [%clk 0:01:00]>
1	https://lichess.org/brBOPdLj	e4 d5 { [%clk 0:01:00]>
2	https://lichess.org/pk0m4lj8	g3 c6 { [%clk 0:01:00]>
3	https://lichess.org/ZXitDcRa	e4 d6 { [%clk 0:01:00]>
4	https://lichess.org/j50FcHUm	e4 e5 { [%clk 0:01:00]>'''

In [ ]:
# Searched for '2.' which indicated the second move and removed it and everything after it. 

df['First Move'] = df['First Move'].str.replace(r'2\..*', '', regex=True)


In [ ]:
''' Searched for a pattern that would remove '{ [%clk 0:01:00]>' and similar text. 
Now only left with moves for the most part. More cleaning needed but will be handled later on.'''

df['First Move'] = df['First Move'].astype(str)
pattern = re.compile(r'\{.*?\>')

# apply the pattern to the 'moves' column and replace with an empty string
df['First Move'] = df['First Move'].apply(lambda x: pattern.sub('', x))

print(df['First Move'])

'''Left with moves and Game URLs'''

df.head(5)
'''
Game URL	First Move
0	https://lichess.org/aAcMUViA	e4 g6
1	https://lichess.org/brBOPdLj	e4 d5
2	https://lichess.org/pk0m4lj8	g3 c6
3	https://lichess.org/ZXitDcRa	e4 d6
4	https://lichess.org/j50FcHUm	e4 e5'''

__Notebook 3: Combining First Move and URL__

In [ ]:
import pandas as pd

In [ ]:
# Load data frames
df0 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_0')
df1 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_1')
df2 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_2')
df3 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_3')
df4 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_4')
df5 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_5')
df6 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_6')
df7 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_7')
df8 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_8')
df9 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_9')
df10 = pd.read_csv(r'C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_10')

In [ ]:
# Combine the data frames vertically
df = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], axis=0)

In [ ]:
print(df10)
'''         Unnamed: 0                      Game URL First Move
0                 0  https://lichess.org/r58eoBfT  e4  1-0'>
1                 1  https://lichess.org/nmsP3uSt    d4  d5 
2                 2  https://lichess.org/MGstATaJ    e4  d5 
3                 3  https://lichess.org/ZJ1AAfU3   Nf3  d5 
4                 4  https://lichess.org/8gfBl9tS    e4  c6 
...             ...                           ...        ...
2670434     2670434  https://lichess.org/TYiG82Ml    e4  d5 
2670435     2670435  https://lichess.org/rauLSxh4    e4  e5 
2670436     2670436  https://lichess.org/ZJq2TAOR    e4  *'>
2670437     2670437  https://lichess.org/SBzVsuns    e4  e5 
2670438     2670438  https://lichess.org/03o6ZYeQ    e4  e5 

[2670439 rows x 3 columns]'''

In [ ]:
print(df0.shape, df1.shape, df2.shape, df3.shape, df4.shape, df5.shape, df6.shape, df7.shape, df8.shape, df9.shape, df10.shape)
'''(1, 3) (10000000, 3) (10000000, 3) (10000000, 3) (10000000, 3) (10000000, 3) (10000000, 3) (10000000, 3) (10000000, 3) (10000000, 3) (2670439, 3)'''

In [ ]:
df.head
'''
Unnamed: 0	Game URL	First Move
0	0	https://lichess.org/aAcMUViA	e4 g6
0	0	https://lichess.org/brBOPdLj	e4 d5
1	1	https://lichess.org/pk0m4lj8	g3 c6
2	2	https://lichess.org/ZXitDcRa	e4 d6
3	3	https://lichess.org/j50FcHUm	e4 e5'''

In [ ]:
df.info()
'''<class 'pandas.core.frame.DataFrame'>
Int64Index: 92670440 entries, 0 to 2670438
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Unnamed: 0  int64 
 1   Game URL    object
 2   First Move  object
dtypes: int64(1), object(2)
memory usage: 2.8+ GB'''


In [ ]:
df.sample(10)
'''	Unnamed: 0	Game URL	First Move
3003692	3003692	https://lichess.org/uBSsHQyF	e4 c5
621402	621402	https://lichess.org/p5skQzPm	c4 e5
4609952	4609952	https://lichess.org/0VEXbv2m	e4 c5
1802213	1802213	https://lichess.org/HMn7YVcN	d4 f5
7506627	7506627	https://lichess.org/FC2xoer5	e4 c5
4505837	4505837	https://lichess.org/nEzdPMZc	e4 a5?!
67858	67858	https://lichess.org/JcjyGmbQ	e4 d5
8135403	8135403	https://lichess.org/XHzxedRl	e4 1-0'>
670720	670720	https://lichess.org/ou2V3m9a	e4 e5
2004260	2004260	https://lichess.org/ZERBOi7j	a4 d5'''

__Notebook 4: Compiling all data into one dataframe__

In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
df1 = pd.read_csv(r"C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_dataframe_Data_Definition.csv", index_col=[0])
df2 = pd.read_csv(r"C:\Users\PC\Desktop\Springboard Data Science Career Track\Springboard - github\DataScience Capstone 2\lichess_first_move_urls\lichess_first_move_urls_all", index_col=[0])                             

In [ ]:
# merged df2()
df = pd.merge(df1, df2, on='Game URL')

# take a look at df.head
df.head()
'''

Event	White Elo	Black Elo	Result	Termination	Time Control	Opening	Game URL	Unnamed: 0	First Move
0	Rated Blitz game	1429	1460	1-0	Normal	300+3	Sicilian Defense: Bowdler Attack	https://lichess.org/bVgQVZ7L	12	e4 c5
1	Rated Blitz game	1687	1647	1-0	Time forfeit	300+3	King's Pawn Game: Leonardis Variation	https://lichess.org/2ORvpVTt	13	e4 e5
2	Rated Blitz game	1778	1837	1-0	Normal	300+3	Queen's Gambit Declined: Neo-Orthodox Variatio...	https://lichess.org/rNNyi2Xh	14	d4 d5
3	Rated Blitz game	1101	1086	0-1	Normal	300+3	Queen's Gambit Declined: Marshall Defense	https://lichess.org/cLhZKY8t	15	d4 d5
4	Rated Blitz game	1362	1408	0-1	Normal	300+3	Sicilian Defense: Smith-Morra Gambit	https://lichess.org/aVEa6ISS	16	e4 c5'''

In [ ]:
# Splitting "First Move" into White's and Black's First moves
df[['White First Move','Black First Move']] = df['First Move'].str.split(' ', n=1, expand=True)

In [ ]:
df['White First Move'].unique()
'''array(['e4', 'd4', 'f4', 'd3', 'Nf3', 'e3', 'b4', 'b3', 'c4', 'None',
       'Nc3', 'g3', 'h3', 'f3', 'Nh3', 'c3', 'g4', 'h4', 'g4?', 'a4',
       'a3', 'f3?!', 'h4?!', 'Na3', 'Nh3?!', 'Na3?!'], dtype=object)'''

In [ ]:
# # Replace "?" and "!" with 'None'. They signify analysis of the move.

df['White First Move'] = df['White First Move'].str.replace(re.escape('?'), '') \
                                                .str.replace(re.escape('!'), '') 

In [ ]:
df['White First Move'].unique()
'''array(['e4', 'd4', 'f4', 'd3', 'Nf3', 'e3', 'b4', 'b3', 'c4', 'None',
       'Nc3', 'g3', 'h3', 'f3', 'Nh3', 'c3', 'g4', 'h4', 'a4', 'a3',
       'Na3'], dtype=object)'''

In [ ]:
df['Black First Move'].unique()
'''array([' c5 ', ' e5 ', ' d5 ', ' c6 ', ' Nf6 ', ' b6 ', ' Nc6 ', ' e6 ',
       ' d6 ', ' g6 ', None, " 1-0'>", ' f5 ', ' h5 ', ' d6?! ', ' c5?! ',
       ' b5 ', ' g5 ', ' e5? ', ' g5? ', ' f6 ', ' a6 ', ' a5 ', ' h6 ',
       ' b6?! ', ' f5?! ', ' Na6 ', ' b5? ', ' g6?! ', ' f5? ', ' h5? ',
       ' f5?! >', ' d6?!>', ' Nc6?! ', ' Nh6 ', ' f6? >', ' Nf6 >',
       ' a5? >', ' f6? ', ' h5?! ', ' d5 >', ' f6?! ', ' e5 >', ' a6?! ',
       ' b5?! ', ' f6?! >', ' Nh6? ', ' d6?>', ' Nc6>', ' a6? >', ' Nf6>',
       ' b6?>', ' e6 >', ' a5?>', ' h6?! ', ' a5?! ', ' Nc6 >',
       ' Na6?! >', " 0-1'>", ' Na6? ', ' c5 >', ' g5? >', ' b6? >',
       ' e5>', ' g6? >', ' d5>', ' c5>', ' e6?!>', ' c5?!>', ' a5?! >',
       ' g6?!>', ' c6?!>', ' f5>', ' Nf6?>', ' f5?!>', ' f5?>', ' Nh>',
       ' Nh6? >', ' h5?! >', ' g5?! ', ' h5? >', ' c6?>', ' c6>', ' Na>',
       ' a6?! >', ' f6?>', ' a6?!>', ' Nf>', ' h5?>', ' a6?>', ' g6?>',
       ' d6>', ' Nh6?! >', ' h6?>', ' h5>', ' h6>', ' g6>', ' f5? >',
       ' b5? >', ' h6? >', ' h5?!>', ' e6>', ' g5?>', ' b6>', ' b5?>',
       ' Na6?!>', ' Nh6?>', ' Na6?>', ' Nc>', ' Nh6?! ', ' a6>', ' b5>',
       ' Na6>', ' Nh6>', ' a5>', ' g5>', ' h6?!>', ' Nh6?!>',
       " 1/2-1/2'>", ' Na6?! ', ' f6>', ' h6 >', ' c6 >', ' d6 >',
       ' a6 >'], dtype=object)'''

In [ ]:
# Replace "?", "!", ">'" with 'None'. The first two signify analysis of the move 
# and the last one is an extra character. 

df['Black First Move'] = df['Black First Move'].str.replace(re.escape('?'), '') \
                                                .str.replace(re.escape('!'), '') \
                                                .str.replace(re.escape(' '), '') \
                                                .str.replace(re.escape('>'), '')

In [ ]:
df['Black First Move'].unique()
'''
array(['c5', 'e5', 'd5', 'c6', 'Nf6', 'b6', 'Nc6', 'e6', 'd6', 'g6', None,
       "1-0'", 'f5', 'h5', 'b5', 'g5', 'f6', 'a6', 'a5', 'h6', 'Na6',
       'Nh6', "0-1'", 'Nh', 'Na', 'Nf', 'Nc', "1/2-1/2'"], dtype=object)'''

In [ ]:
# Replace "1-0'", "0-1'", "1/2-1/2'" which signify end of a game with 'None'

df['Black First Move'] = df['Black First Move'].str.replace(re.escape("1-0'"), 'None') \
                                                .str.replace(re.escape("0-1'"), 'None') \
                                                .str.replace(re.escape("1/2-1/2'"), 'None') 
df['Black First Move'].unique()    

'''array(['c5', 'e5', 'd5', 'c6', 'Nf6', 'b6', 'Nc6', 'e6', 'd6', 'g6', None,
       'None', 'f5', 'h5', 'b5', 'g5', 'f6', 'a6', 'a5', 'h6', 'Na6',
       'Nh6', 'Nh', 'Na', 'Nf', 'Nc'], dtype=object)'''

In [ ]:
# "Na", "Nh", "Nc", "Nf". Those seem strange. 
# It appears that the moves were either partially recorded. 
# Fortunately, this is easily fixed as there is only one choice for these two. 
# The moves have to be “Na6”, “Nh6”, "Nc6", "Nf6" respectively. 
df['Black First Move'] = df['Black First Move'].str.replace(re.escape("Na6"), 'Na') \
                                                .str.replace(re.escape("Na"), 'Na6') \
                                                .str.replace(re.escape("Nh6"), 'Nh') \
                                                .str.replace(re.escape("Nh"), 'Nh6') \
                                                .str.replace(re.escape("Nf6"), 'Nf') \
                                                .str.replace(re.escape("Nf"), 'Nf6') \
                                                .str.replace(re.escape("Nc6"), 'Nc') \
                                                .str.replace(re.escape("Nc"), 'Nc6')

In [ ]:
# No more "Na", "Nh", "Nc", "Nf"

df4 = len(df[(df['Black First Move']=='Nh') & (df['Black First Move']!='Nh6')])
df5 = len(df[(df['Black First Move']=='Na') & (df['Black First Move']!='Na6')])
df6 = len(df[(df['Black First Move']=='Nc') & (df['Black First Move']!='Nc6')])
df7 = len(df[(df['Black First Move']=='Nf') & (df['Black First Move']!='Nf6')])
print(df4, df5, df6, df7)

In [ ]:
# Count NaN values in the DataFrame
n_missing = df['Black First Move'].isna().sum().sum()

# Print the number of NaN values
print("Number of NaN values: ", n_missing)
'''Number of NaN values:  96581'''

In [ ]:
df['Black First Move'] = df['Black First Move'].fillna('None')
'''Number of unique values in `White First Move`:  21
List of unique values in `White First Move`:  ['e4' 'd4' 'f4' 'd3' 'Nf3' 'e3' 'b4' 'b3' 'c4' 'None' 'Nc3' 'g3' 'h3' 'f3'
 'Nh3' 'c3' 'g4' 'h4' 'a4' 'a3' 'Na3']
Number of unique values in `Black First Move`:  21
List of unique values in `Black First Move`:  ['c5' 'e5' 'd5' 'c6' 'Nf6' 'b6' 'Nc6' 'e6' 'd6' 'g6' 'None' 'f5' 'h5' 'b5'
 'g5' 'f6' 'a6' 'a5' 'h6' 'Na6' 'Nh6']'''

In [ ]:
# Getting rid of "First Move" column since we now have "White First Move" & "Black First Move".

df = df[['Event', 'White Elo', 'Black Elo', 'Result', 'Termination', 'Time Control', 'Opening',"Game URL", "White First Move", "Black First Move"]]      
df.head()

'''Event	White Elo	Black Elo	Result	Termination	Time Control	Opening	Game URL	White First Move	Black First Move
0	Rated Blitz game	1429	1460	1-0	Normal	300+3	Sicilian Defense: Bowdler Attack	https://lichess.org/bVgQVZ7L	e4	c5
1	Rated Blitz game	1687	1647	1-0	Time forfeit	300+3	King's Pawn Game: Leonardis Variation	https://lichess.org/2ORvpVTt	e4	e5
2	Rated Blitz game	1778	1837	1-0	Normal	300+3	Queen's Gambit Declined: Neo-Orthodox Variatio...	https://lichess.org/rNNyi2Xh	d4	d5
3	Rated Blitz game	1101	1086	0-1	Normal	300+3	Queen's Gambit Declined: Marshall Defense	https://lichess.org/cLhZKY8t	d4	d5
4	Rated Blitz game	1362	1408	0-1	Normal	300+3	Sicilian Defense: Smith-Morra Gambit	https://lichess.org/aVEa6ISS	e4	c5'''